In [1]:
from segment_anything.build_sam import  *
from segment_anything.automatic_mask_generator import SamAutomaticMaskGenerator
from segment_img import *
# from general_inference import convert
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np
import os
import json
import time

In [4]:
img_path = '/root/data_preprocessing/box_labeled/test_data_croped'
checkpoint_path = '/root/autodl-fs/pretrain_UniLSeg/sam_vit_h_4b8939.pth'
# checkpoint_l = '/Users/mtsbx/PycharmProjects/UniLSeg/data_preprocessing/segment-anything/pretrain/sam_vit_l_0b3195.pth'

input_dir = img_path
output_dir = '/root/data_preprocessing/box_labeled/test_data_sam'
json_output_path = os.path.join(output_dir, "results.json")

sam_model = build_sam_vit_h(checkpoint_path)
mask_generator  = SamAutomaticMaskGenerator(sam_model)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# load_model
sam_model = build_sam_vit_h(checkpoint_path)
mask_generator = SamAutomaticMaskGenerator(sam_model)

print("\n" + "=" * 20 + " 模型加载成功 " + "=" * 20 + "\n")

all_results = []

# 遍历输入目录中所有图片文件
for filename in os.listdir(input_dir):
    i = 0
    i += 1
    if i > 1:
        break
    if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
        img_path = os.path.join(input_dir, filename)
        start_time = time.time()
        # 读取图像（OpenCV 返回 BGR 格式）
        img_bgr = cv2.imread(img_path)
        if img_bgr is None:
            print(f"无法加载图像：{img_path}")
            continue
        # 转换为 RGB 格式（SAM 模型一般使用 RGB 格式）
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        # 生成 mask，输入要求为 HWC, uint8 格式的 NumPy 数组
        masks = mask_generator.generate(img_rgb)

        # 保存每个 mask 的二值图像，同时记录数值信息
        base_name = os.path.splitext(filename)[0]
        image_results = {
            "image": filename,
            "masks": []
        }
        for i, mask_info in enumerate(masks):
            seg = mask_info["segmentation"]
            seg = ensure_mask_2d(seg)
            # 将布尔型 mask 转换为 0-255 的 uint8 图像
            seg_img = (seg.astype(np.uint8)) * 255
            mask_filename = f"{base_name}_mask_{i}.png"
            out_path = os.path.join(output_dir, mask_filename)
            cv2.imwrite(out_path, seg_img)

            # 保存该 mask 的数值信息
            mask_data = {
                "mask_file": mask_filename,
                "bbox": mask_info["bbox"],  # [x, y, w, h]
                "area": mask_info["area"],
                "predicted_iou": mask_info["predicted_iou"],
                "stability_score": mask_info["stability_score"],
                "point_coords": mask_info["point_coords"],
                "crop_box": mask_info["crop_box"]
            }
            image_results["masks"].append(mask_data)

        all_results.append(image_results)
        elapsed_time = time.time() - start_time
        print(f"处理完成：{filename}，耗时 {elapsed_time:.2f} 秒")

# 将所有结果保存到 JSON 文件中
with open(json_output_path, "w") as f:
    json.dump(all_results, f, indent=4)
print(f"所有分割结果保存到：{json_output_path}")


==================== 模型加载成功 ====================

处理完成：COCO_train2014_000000098304_ann003007.jpg，耗时 20.43 秒
处理完成：COCO_train2014_000000098304_ann099893.jpg，耗时 19.96 秒
处理完成：COCO_train2014_000000098304_ann108703.jpg，耗时 19.99 秒
处理完成：COCO_train2014_000000098304_ann115415.jpg，耗时 21.33 秒
处理完成：COCO_train2014_000000098304_ann116865.jpg，耗时 19.74 秒
处理完成：COCO_train2014_000000098304_ann120114.jpg，耗时 21.12 秒
处理完成：COCO_train2014_000000098304_ann1490146.jpg，耗时 20.18 秒
处理完成：COCO_train2014_000000098304_ann1511501.jpg，耗时 21.30 秒
处理完成：COCO_train2014_000000098304_ann1513367.jpg，耗时 21.19 秒
处理完成：COCO_train2014_000000098304_ann2099764.jpg，耗时 20.78 秒
处理完成：COCO_train2014_000000098304_ann2222700.jpg，耗时 19.58 秒
处理完成：COCO_train2014_000000098304_ann2222956.jpg，耗时 20.46 秒
所有分割结果保存到：/root/data_preprocessing/box_labeled/test_data_sam/results.json


In [7]:
import os
import time
import random
import json

import cv2
import numpy as np
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from segment_anything import build_sam_vit_h, SamAutomaticMaskGenerator

def overlay_and_save_masks(
    input_dir: str,
    checker_json: str,
    checkpoint_path: str,
    output_dir: str,
    alpha: float = 0.5,
    max_images: int = None
):
    os.makedirs(output_dir, exist_ok=True)

    # 1. 加载 SAM 模型
    sam = build_sam_vit_h(checkpoint_path)
    mask_gen = SamAutomaticMaskGenerator(sam)

    # 2. 如果提供了 COCO JSON，则只处理其中的文件
    valid_fnames = None
    if checker_json:
        coco = COCO(checker_json)
        valid_fnames = {info['file_name'] for info in coco.loadImgs(coco.getImgIds())}

    # 3. 遍历目录
    for idx, fname in enumerate(sorted(os.listdir(input_dir))):
        if max_images and idx >= max_images:
            break
        if not fname.lower().endswith(('.jpg','.png','jpeg','bmp')):
            continue
        if valid_fnames and fname not in valid_fnames:
            continue

        img_path = os.path.join(input_dir, fname)
        img_bgr  = cv2.imread(img_path)
        if img_bgr is None:
            print(f"[Warning] 无法加载 {img_path}")
            continue
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        # 4. 生成掩码列表
        masks = mask_gen.generate(img_rgb)

        # 5. 叠加
        overlay = img_rgb.copy()
        for m in masks:
            seg = m["segmentation"]

            # 如果是 RLE dict，先解码
            if isinstance(seg, dict):
                seg = maskUtils.decode(seg)  # returns HxW ndarray of {0,1}

            # 如果多通道，把它 squeeze 到 2D
            if isinstance(seg, np.ndarray) and seg.ndim > 2:
                seg = np.squeeze(seg)

            # seg 现在应该是 HxW 布尔/0-1 数组
            mask_bool = seg.astype(bool)

            # 随机颜色
            color = [random.randint(0,255) for _ in range(3)]
            # alpha 混合
            overlay[mask_bool] = (
                overlay[mask_bool] * (1 - alpha) +
                np.array(color) * alpha
            ).astype(np.uint8)

        # 6. 转回 BGR 并保存
        out_bgr = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)
        save_name = f"{os.path.splitext(fname)[0]}_overlay.jpg"
        save_path = os.path.join(output_dir, save_name)
        cv2.imwrite(save_path, out_bgr)
        print(f"[Saved] {save_path} （{len(masks)} masks）")


if __name__ == "__main__":
    # input_dir      = '/root/data_preprocessing/box_labeled/test_data_croped'
    input_dir      = '/root/data_preprocessing/box_labeled/sam_test'
    checker_json   = None   # 或者你的 'instances_filtered.json'
    checkpoint     = '/root/autodl-fs/pretrain_UniLSeg/sam_vit_h_4b8939.pth'
    # output_dir     = '/root/data_preprocessing/box_labeled/test_data_sam_overlay'
    output_dir     = '/root/data_preprocessing/box_labeled/sam_test'
    
    overlay_and_save_masks(
        input_dir=input_dir,
        checker_json=checker_json,
        checkpoint_path=checkpoint,
        output_dir=output_dir,
        alpha=0.5,
        max_images=None  # 全部图片
    )

[Saved] /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_overlay.jpg （59 masks）


In [4]:
import os
import cv2
import numpy as np
from segment_anything import build_sam_vit_h, SamAutomaticMaskGenerator
from pycocotools import mask as maskUtils

def segment_and_save(
    img_path: str,
    checkpoint_path: str,
    output_dir: str,
    model_type: str = "vit_h",  # "vit_h" 对应 checkpoint_h, 若用 vit_l 则加载相应权重
    pick_largest: bool = True
):
    """
    对单张图片做 SAM 分割，选取面积最大的 mask，保存：
      - 原图同尺寸的二值掩码：mask.png
      - 只含目标的 RGB 抠图：obj.png
    
    Args:
        img_path:        待分割的图片路径
        checkpoint_path: SAM 模型权重文件路径
        output_dir:      输出目录，会自动创建
        model_type:      "vit_h" 或 "vit_l"（名称要与你的 checkpoint 一致）
        pick_largest:    是否只保留面积最大的一个 mask
    """
    os.makedirs(output_dir, exist_ok=True)

    # 1. 加载 SAM 与 mask generator
    sam = build_sam_vit_h(checkpoint_path) if model_type=="vit_h" \
          else build_sam_vit_h(checkpoint_path, model_type="vit_l")
    mask_gen = SamAutomaticMaskGenerator(sam)

    # 2. 读取图像并转 RGB
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        raise FileNotFoundError(f"无法加载图片: {img_path}")
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # 3. 调用 SAM 生成所有 mask
    masks = mask_gen.generate(img_rgb)
    if not masks:
        print("⚠️ 未生成任何 mask，跳过。")
        return

    # 4. 挑一个 mask：要么最大的，要么所有
    if pick_largest:
        mask_info = max(masks, key=lambda m: m["area"])
        mask_list = [mask_info["segmentation"]]
    else:
        mask_list = [m["segmentation"] for m in masks]

    # 5. 处理并保存
    base = os.path.splitext(os.path.basename(img_path))[0]
    for idx, seg in enumerate(mask_list):
        # RLE 解码
        if isinstance(seg, dict):
            seg = maskUtils.decode(seg)
        # squeeze 到 2D
        if isinstance(seg, np.ndarray) and seg.ndim > 2:
            seg = np.squeeze(seg)
        mask_bool = seg.astype(bool)

        # 保存二值掩码图
        mask_img = (mask_bool.astype(np.uint8)) * 255
        mask_path = os.path.join(output_dir, f"{base}_mask_{idx}.png")
        cv2.imwrite(mask_path, mask_img)
        print(f"[Saved] mask: {mask_path}")

        # 抠图：只保留前景
        obj_rgb = img_rgb.copy()
        obj_rgb[~mask_bool] = 255  # 背景置白，方便查看
        obj_bgr = cv2.cvtColor(obj_rgb, cv2.COLOR_RGB2BGR)
        obj_path = os.path.join(output_dir, f"{base}_obj_{idx}.png")
        cv2.imwrite(obj_path, obj_bgr)
        print(f"[Saved] object crop: {obj_path}")


if __name__ == "__main__":
    # —— 用户配置 —— 
    input_image     = "/root/data_preprocessing/box_labeled/test_data_croped/COCO_train2014_000000098304_ann116865.jpg"
    sam_checkpoint  = "/root/autodl-fs/pretrain_UniLSeg/sam_vit_h_4b8939.pth"
    output_folder   = "/root/data_preprocessing/box_labeled/sam_test"
    model_variant   = "vit_h"   # 或者 "vit_l"（若你下载了 ViT-L checkpoint）

    segment_and_save(
        img_path=input_image,
        checkpoint_path=sam_checkpoint,
        output_dir=output_folder,
        model_type=model_variant,
        pick_largest=False
    )

[Saved] mask: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_mask_0.png
[Saved] object crop: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_obj_0.png
[Saved] mask: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_mask_1.png
[Saved] object crop: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_obj_1.png
[Saved] mask: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_mask_2.png
[Saved] object crop: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_obj_2.png
[Saved] mask: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_mask_3.png
[Saved] object crop: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_obj_3.png
[Saved] mask: /root/data_preprocessing/box_labeled/sam_test/COCO_train2014_000000098304_ann116865_mask_4